In [1]:
import sqlite3
import pandas as pd
import numpy as np
con=sqlite3.connect('./amazon-fine-food-reviews/database.sqlite')
filtereddata=pd.read_sql_query("""SELECT * FROM Reviews WHERE Score != 3""",con)
def partition(x):
    if x<3:
        return 'negetive'
    return 'positive'

actualscore=filtereddata['Score']
filtereddata['Score']=actualscore.map(partition)
filtereddata.shape
filtereddata.head()



,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negetive,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negetive,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
data=pd.read_sql_query("""SELECT * FROM Reviews WHERE Score!=3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID""",con)
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [3]:
sorteddata=filtereddata.sort_values('ProductId',axis=0,ascending=True)

In [4]:
final=sorteddata.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

In [5]:
display=pd.read_sql_query("""SELECT * FROM Reviews WHERE Score!=3 AND HelpfulnessDenominator < HelpfulnessNumerator ORDER BY ProductId""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [6]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [7]:
final.shape
#final['Score'].value_counts() # count values 

(364171, 10)

In [8]:
## bag of words (BOW)

from sklearn.feature_extraction.text import CountVectorizer
countvect=CountVectorizer()
finalcount=countvect.fit_transform(final['Text'].values)


In [9]:
type(finalcount)

scipy.sparse.csr.csr_matrix

In [10]:
finalcount.shape

(364171, 115281)

In [22]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentences):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentences)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
print(stop)
print('*****************************')
print(sno.stem('tasty'))

{"isn't", 'ours', 'when', 'its', 'my', 'both', 'more', 'same', 'out', 'needn', 'been', 'most', 'here', 'now', 't', 'up', 'they', 'and', 'very', 'weren', 'from', 'ain', 'should', 'mightn', "hasn't", 'being', 'some', 'll', 'having', 'have', 'over', 'ma', 'isn', 'these', 'until', "you've", 'we', 'him', 'nor', 'do', 'didn', 'our', 'does', 'were', 'y', "doesn't", 'ourselves', "didn't", 'herself', "you'll", "haven't", "mustn't", 'has', 'so', "won't", 'again', 'just', 'who', 'with', 've', 'doesn', 'you', 'whom', 'why', "that'll", 'because', 'an', 'hers', 'did', 'any', 'each', 'there', 'those', "wouldn't", 'of', 'that', 'once', 'was', 'hadn', 'own', 's', 'i', 'but', 'theirs', 'themselves', 'a', 'by', 'for', 'shouldn', 'or', 'at', 'her', 'this', 'such', 'their', 'your', "should've", 'below', 'how', 'yourselves', 'his', 'aren', 'only', 'don', 'above', 'mustn', 'not', 'couldn', 'the', 'm', 'if', "you're", 'can', 'me', 'while', 'it', 'after', 'wasn', 'itself', "don't", 'under', 'she', 'shan', 'wou

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [ ]:
i=0
strl=' '
final_string=[]
all_positive_words=[]
all_negetive_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i] == 'negative':
                        all_negetive_words.append(s)
                else:
                    continue
            else:
                continue
                    
    strl = b" ".join(filtered_sentence)
    
    final_string.append(strl)
    i+=1 

In [ ]:
final['CleanedText']=final_string

In [ ]:
final.head(3)

conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn,flavor=None,schema=None,if_exists='replace')

In [22]:
from sklearn.feature_extraction.text import CountVectorizer


count_vect=CountVectorizer(ngram_range=(1,2))
final_bigram_counts=count_vect.fit_transform(final['Text'].values)

In [23]:
final_bigram_counts.get_shape()

(364171, 2910192)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(ngram_range=(1,2))
finaltfidf=tfidf.fit_transform(final['Text'].values)
finaltfidf.get_shape()

(364171, 2910192)

In [27]:
feature=tfidf.get_feature_names()

In [30]:
feature[100:110]

['00 cans',
 '00 cappuccino',
 '00 capsules',
 '00 carton',
 '00 case',
 '00 cd',
 '00 chai',
 '00 chance',
 '00 change',
 '00 cheaper']

In [31]:
print(finaltfidf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [42]:
def topidf(row,feature,top_n=25):
    topid=np.argsort(row)
    top_feats=[(feature[i],row[i])for i in topid]
    df=pd.DataFrame(top_feats)
    df.columns=['feature','tfidf']
    return df
toptfidf=topidf(finaltfidf[1,:].toarray()[0],feature,25)

In [43]:
toptfidf

,feature,tfidf
0,00,0.000000
1,poor think,0.000000
2,poor this,0.000000
3,poor though,0.000000
4,poor thought,0.000000
5,poor three,0.000000
6,poor to,0.000000
7,poor tobias,0.000000
8,poor tolerance,0.000000
9,poor things,0.000000


In [49]:
!pip install gensim


The following command must be run outside of the IPython shell:

    $ pip install gensim

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [53]:
import gensim
i=0
listofsent=[]
for sent in final['Text'].values:
    filteredsentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleanword in cleanpunc(w).split():
            if(cleanword.isalpha()):
                filteredsentence.append(cleanword.lower())
            else:
                continue;
    listofsent.append(filteredsentence)

ModuleNotFoundError: No module named 'gensim'

In [60]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn import datasets
import pandas as pd
import matplotlib.pyplot as plt
model=TSNE(n_components=2,random_state=0)
tsne_data=model.fit_transform(final['Text'].values)

ValueError: could not convert string to float: "this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college"